In [109]:
params = {}

In [110]:
import time
import json
import sys

from services.config.service.config_directories import directories, get_directories
from services.dte_tools.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, post_file_to_duft_server  # noqa: E402

In [111]:
   
environment = initialise_data_task("Jupyter Sample Data Upload Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))

if not params:
    environment.log_error("No parameters given!")
    

[2025-04-22T14:12:21.048716] Jupyter Sample Data Upload Task - INFO: Jupyter Sample Data Upload Task initialised running in active mode
Jupyter Sample Data Upload Task >>>: Jupyter Sample Data Upload Task initialised running in active mode


In [112]:
assert_dte_tools_available()

resolved_parameters = get_resolved_parameters_for_connection("ANA")
resolved_server_api_parameters = get_resolved_parameters_for_connection("DUFT-SERVER-API")
tmp_dir = directories.data_tasks_temp_directory
data_dir = directories.data_directory
environment.log_message('Data Upload Starting!')
environment.log_message("Using parameters: %s" % resolved_parameters)
environment.log_message("Using temp directory: %s" % tmp_dir)
    
print(resolved_server_api_parameters)

DTE Tools available
[2025-04-22T14:12:21.056661] Jupyter Sample Data Upload Task - INFO: Data Upload Starting!
Jupyter Sample Data Upload Task >>>: Data Upload Starting!
[2025-04-22T14:12:21.057641] Jupyter Sample Data Upload Task - INFO: Using parameters: {'server': '127.0.0.1', 'username': 'user', 'password': 'mysecretword', 'port': '5432', 'database': 'analysis', 'sqlite3file': 'wakanda.sqlite', 'type': 'sqlite3'}
Jupyter Sample Data Upload Task >>>: Using parameters: {'server': '127.0.0.1', 'username': 'user', 'password': 'mysecretword', 'port': '5432', 'database': 'analysis', 'sqlite3file': 'wakanda.sqlite', 'type': 'sqlite3'}
[2025-04-22T14:12:21.058462] Jupyter Sample Data Upload Task - INFO: Using temp directory: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks
Jupyter Sample Data Upload Task >>>: Using temp directory: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks
{'server': 'http://127.0.0.1:8000'

In [113]:
# Load sample dataset from SQL File

import pandas as pd
import sqlite3
import os

# Load the data
conn = sqlite3.connect(os.path.join(data_dir, resolved_parameters['sqlite3file']))
data = pd.read_sql_query("SELECT * FROM dim_client LIMIT 20", conn)
conn.close()

file_to_save = os.path.join(tmp_dir, 'dim_client_sample.csv')
data.to_csv(file_to_save, index=False)

data.head()



,client_id,emr_identifier,patient_pkv,facility_number,pbs_scenario,his_entry_type,number_h,birth_date,gender,death_date,...,is_disabled,disability,residence_ward,residence_village,residence_sub_county,residence_district,residence_county,residence_region,residence_country,date_imported_emr
0,1025682,1111130,,1025698,,EMR,,1960-12-24,Female,,...,,,,,,,,,Wakanda,
1,1129715,1215163,,1129731,,EMR,,1982-04-09,Female,,...,,,,,,,,,Wakanda,
2,1132571,1218019,,1132587,,EMR,,1983-06-15,Female,,...,,,,,,,,,Wakanda,
3,1132573,1218021,,1132589,,EMR,,1979-10-14,Male,,...,,,,,,,,,Wakanda,
4,1132575,1218023,,1132591,,EMR,,2008-10-08,Male,,...,,,,,,,,,Wakanda,


In [114]:
try:
    post_file_to_duft_server(
        resolved_server_api_parameters["server"], 
        resolved_server_api_parameters["username"], 
        resolved_server_api_parameters["password"], 
        file_to_save, 
        "client-list")
    
    environment.log_message('Data Upload Complete!')

except Exception as e:
    environment.log_error(f"Failed to upload file: {e}")



[2025-04-22T14:12:21.612792] Jupyter Sample Data Upload Task - ERROR: Failed to authenticate: 401 - {"detail":"No active account found with the given credentials"}
[2025-04-22T14:12:21.614127] Jupyter Sample Data Upload Task - ERROR: Failed to upload file: Failed to authenticate: 401 - {"detail":"No active account found with the given credentials"}
Jupyter Sample Data Upload Task >>>: ERROR: Failed to upload file: Failed to authenticate: 401 - {"detail":"No active account found with the given credentials"}


In [115]:
if os.path.exists(file_to_save):
    os.remove(file_to_save)
    environment.log_message(f"Removing data file: {file_to_save}")
else:
    environment.log_message(f"File does not exist: {file_to_save}")

[2025-04-22T14:12:21.621871] Jupyter Sample Data Upload Task - INFO: Removing data file: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks/dim_client_sample.csv
Jupyter Sample Data Upload Task >>>: Removing data file: /Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/tmp-data-tasks/dim_client_sample.csv


In [116]:

environment.log_message('Data Upload Completed!')

[2025-04-22T14:12:21.626360] Jupyter Sample Data Upload Task - INFO: Data Upload Completed!
Jupyter Sample Data Upload Task >>>: Data Upload Completed!
